# Aufgabe 1:

In [15]:
import numpy
import math
from scipy.integrate import quad

def trapez_integration(f, a, b, n_trapez):
    h = (b - a) / n_trapez
    s = 0.5 * (f(a) + f(b))
    for i in range(1, n_trapez):
        s += f(a + i * h)
    return s * h


def simpson_integration(f, a, b, n_simpson):
    h = (b - a) / n_simpson
    s = f(a) + f(b)
    for i in range(1, n_simpson):
        if i % 2 == 0:
            s += 2 * f(a + i * h)
        else:
            s += 4 * f(a + i * h)
    return (s - f(a) - f(b)) * (h / 3)


def g2_integration(f, a, b):
    xs = numpy.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
    betas = numpy.array([5/9, 8/9, 5/9])
    xs_ = (b-a)/2 * xs + (b+a)/2
    betas_ = (b-a)/2 * betas
    return numpy.sum(betas_ * f(xs_))


true_value = math.pi/4
trapez_value = trapez_integration(lambda x: 1 / (1+x**2), 0, 1, 8 )
simpson_integration_value = simpson_integration(lambda x: 1 / (1+x**2), 0, 1, 4 )
g2_integration_value = g2_integration(lambda x: 1 / (1+x**2), 0, 1)


print(f"""true value: {true_value}
trapez value: {trapez_value} \t\t error: {abs(true_value - trapez_value)}
simpson value: {simpson_integration_value} \t\t error: {abs(true_value - simpson_integration_value)}
g2 value: {g2_integration_value} \t\t\t error: {abs(true_value - g2_integration_value)}""")



true value: 0.7853981633974483
trapez value: 0.7847471236227722 		 error: 0.0006510397746760654
simpson value: 0.6603921568627451 		 error: 0.12500600653470317
g2 value: 0.7852670349907919 			 error: 0.00013112840665641112


# Aufgabe 2:

In [16]:
def extrapolation_integration(f, a, b, sizes):
    steps = sizes.shape[0]
    ls = 1/sizes
    ls = numpy.asarray(ls)
    ts = numpy.array([trapez_integration(f, a, b, i) for i in sizes])
    ps = numpy.zeros((steps, steps))
    ps[:, 0] = ts
    for i in range(1, steps):
        for j in range(1, i+1):
            ps[i, j] = ps[i, j-1] + (ps[i, j-1] - ps[i-1,j-1])/((ls[i-j]/ls[i])**2-1)
    return ps[steps-1, steps-1]


def romberg_integration(f, a, b, q):
    ls = numpy.array([2**i for i in range(q+1)], dtype=int)
    return extrapolation_integration(f, a, b, ls)


def bulirsch_integration(f, a, b, q):
    ls = [1]
    if q > 1:
        ls.append(2)
    if q > 2:
        ls.append(3)
    if q > 3:
        for i in range(3, q):
            ls.append(ls[i-2]*2)
    ls = numpy.array(ls)
    return extrapolation_integration(f, a, b, ls)


def f(x):
    return math.sin(math.pi * x**2)

true_value = quad(f, -1, 1)[0]

for order in (2,4,6,8,10,12,14,16):
    q = order//2
    print(f"Ordnung: {order}")
    print(f"Romberg: {romberg_integration(f, -1, 1, q)} Fehler: {abs(true_value - romberg_integration(f, -1, 1, q))}")
    print(f"Bulirsch: {bulirsch_integration(f, -1, 1, q)} Fehler: {abs(true_value - bulirsch_integration(f, -1, 1, q))}")
    print("\n")

Ordnung: 2
Romberg: 8.164311994315689e-17 Fehler: 1.0097091882273732
Bulirsch: 2.4492935982947064e-16 Fehler: 1.009709188227373


Ordnung: 4
Romberg: 1.0056629776875345 Fehler: 0.004046210539838668
Bulirsch: 8.164311994315689e-17 Fehler: 1.0097091882273732


Ordnung: 6
Romberg: 1.0250428245891272 Fehler: 0.015333636361754
Bulirsch: 0.9234543869793054 Fehler: 0.08625480124806784


Ordnung: 8
Romberg: 1.0094933959681158 Fehler: 0.00021579225925738577
Bulirsch: 1.1113597371695434 Fehler: 0.10165054894217018


Ordnung: 10
Romberg: 1.009707587534166 Fehler: 1.6006932073153735e-06
Bulirsch: 1.0133666547251936 Fehler: 0.003657466497820394


Ordnung: 12
Romberg: 1.009709194162706 Fehler: 5.935332891837675e-09
Bulirsch: 1.0077702756828084 Fehler: 0.001938912544564797


Ordnung: 14
Romberg: 1.0097091882270657 Fehler: 3.0753177782116836e-13
Bulirsch: 1.0097161455084471 Fehler: 6.957281073916022e-06


Ordnung: 16
Romberg: 1.0097091882273725 Fehler: 6.661338147750939e-16
Bulirsch: 1.009713408769554